In [11]:
from umap.umap_ import UMAP
from bertopic import BERTopic
import numpy as np
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import altair as alt

ImportError: cannot import name 'UMAP' from 'umap' (unknown location)

In [ ]:
# Create an empty list to store processed texts
processed_texts = []
# arrays for extra info like name, year, etc
abstracts = []
filtered_dates = []
filtered_titles = []

# Apply preprocess_text function to each description
for i, description in enumerate(descriptions):
    if not isinstance(description, (float, int)):
        processed = preprocess_text(description)
        processed_texts.append(processed)
        abstracts.append(description)
        if i < len(dates) and i < len(titles):
            filtered_dates.append(dates[i])
            if not isinstance(titles[i], (float, int)):
                filtered_titles.append(titles[i])

# Output the lists to verify correct processing
print("Processed Texts:", processed_texts)
print("Abstracts:", abstracts)
print("Filtered Dates:", filtered_dates)
print("Filtered Titles:", filtered_titles)

In [ ]:
# Create BERTopic instance with desired topic size and automatic number of topics
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)

# embedding model for higher quality
embedding_model = AutoModel.from_pretrained("Salesforce/SFR-Embedding-2_R")

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    nr_topics=15,
)

In [ ]:
# Fit the model on the preprocessed texts
# topics stores topics assigned to documents
topics, probabilities = topic_model.fit_transform(processed_texts)

# Reduce outliers
new_topics = topic_model.reduce_outliers(processed_texts, topics)

In [ ]:
fig = topic_model.visualize_topics()
fig.show()

In [ ]:
# Get the topic coordinates
fig = topicmodel.visualize_topics()
scatter_data = fig.data[0]

# Ensure coordinates length matches topics length
coordinates_length = min(len(scatter_data["x"]), len(topics))

# Get topic embeddings
topic_embeddings = topic_model.topic_embeddings

# Ensure probs is a 2D array
probs = np.array(probabilities)

# If probs is a 1D array, reshape it to 2D
if probs.ndim == 1:
    probs = probs.reshape(-1, 1)

# Use UMAP to reduce the dimensionality of the topic distributions
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.3, metric="correlation")
umap_embeddings = umap_model.fit_transform(probs)

# Create a DataFrame with individual documents and their UMAP embeddings
data = []
for i in range(len(processed_texts)):
    data.append(
        {
            "topic": topics[i],
            "processed_text": processed_texts[i],
            "abstract": abstracts[i],
            "title": titles[i],
            "date": dates[i],
            "x": umap_embeddings[i, 0],
            "y": umap_embeddings[i, 1],
        }
    )

documents_df = pd.DataFrame(data)

# Enable JSON data transformer
alt.data_transformers.enable("json")

# Create Altair chart
chart = (
    alt.Chart(documents_df)
    .mark_circle()
    .encode(
        x="x:Q",
        y="y:Q",
        color="topic:N",
        tooltip=["topic:N", "title:N", "date:N", "abstract:N"],
    )
    .interactive()
)